Introduction

This python file uses Pandas-Data Manipulation Library, Joblib-Parallel processing library to convert a large data file which contains lexical information about users and their lexical knowledge in English while their native speaking language is Arabic into a desired shape. In the final file the columns represent the words, and rows indicate the number of users who has accuratley answered the words presented to them with no less than three errors.

The required information of the users is distributed in three files namely profiles.csv,sessions.csv, lexical-decision.csv.
The column common to any two files is used to combine the user information to create a final entry in the output file.

In [24]:
# Install and import the pandas library
! pip install pandas
import pandas as pd

In [2]:
# Read the profiles file as a csv object.
df_profile = pd.read_csv('D:\\Geoff\profiles.csv')
df_profile.head()

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\gouth\AppData\Local\Temp\ipykernel_4792\605641246.py:1: SyntaxWarning: invalid escape sequence '\p'
  df_profile = pd.read_csv('D:\\Geoff\profiles.csv')


,profile_id,browser_id,ua_id,gender,age,country,education,native_language,level_english,no_foreign_lang,best_foreign,handedness
0,72953,44218,315,Female,32.0,United Arab Emirates,Bachelor,Arabic,I speak and read the language fluently.,0.0,English,Right-handed
1,83529,52259,2853,Female,20.0,Jordan,Bachelor,Arabic,I speak and read the language fluently.,1.0,English,Right-handed
2,96311,61709,55,Male,22.0,Egypt,Bachelor,Arabic,I speak and read the language fluently.,1.0,English,Right-handed
3,117641,77458,7066,Female,41.0,Lebanon,Bachelor,Arabic,I speak and read the language fluently.,1.0,Arabic,Left-handed
4,124298,82277,370,Male,35.0,Saudi Arabia,PhD,Arabic,I speak and read the language fluently.,0.0,Arabic,Right-handed


In [3]:
print(f'Shape of df_profile {df_profile.shape}')
print(f'Number of unique profile ids {len(df_profile['profile_id'].unique())}')

(17559, 12)


17559

In [4]:
# Read the sessions file as csv
df_sessions = pd.read_csv('D:\Geoff\sessions.csv')
df_sessions.head()

<>:1: SyntaxWarning: invalid escape sequence '\G'
<>:1: SyntaxWarning: invalid escape sequence '\G'
C:\Users\gouth\AppData\Local\Temp\ipykernel_4792\1280424540.py:1: SyntaxWarning: invalid escape sequence '\G'
  df_sessions = pd.read_csv('D:\Geoff\sessions.csv')


,exp_id,time_start,time_end,profile_id,ua_id,touch,nw_acc,w_acc,score,profile_id_session
0,50572,29-01-2014 17:01,29-01-2014 17:04,50572,1390,False,0.866667,0.871429,0.738095,1
1,50804,29-01-2014 18:45,29-01-2014 18:50,50572,1390,False,0.933333,0.757143,0.690476,2
2,52177,29-01-2014 23:26,29-01-2014 23:30,52177,3411,False,0.933333,0.442857,0.376190,1
3,53051,29-01-2014 23:42,29-01-2014 23:47,53051,3236,False,0.900000,0.642857,0.542857,1
4,53283,29-01-2014 23:47,29-01-2014 23:50,53283,3479,True,0.866667,0.742857,0.609524,1


In [29]:
print(f'Shape of df_sessions {df_sessions.shape}')
profile_id_count = len(df_sessions['profile_id'].unique())
exp_id_count = len(df_sessions['exp_id'].unique())
print(f"profile_id_count:{profile_id_count}")
print(f"exp_id_count:{exp_id_count}")

#  - (21974, 10)
# Number of unique profile_id's - 17559

Shape of df_sessions (14193, 10)
profile_id_count:11759
exp_id_count:14193


In [30]:
#Filtering the profile_id's who's nw_acc > 93

df_sessions  = df_sessions.loc[df_sessions['nw_acc'] > 0.93]
df_sessions['nw_acc'].value_counts()

nw_acc
1.000000    7203
0.966667    4369
0.933333    2621
Name: count, dtype: int64

In [32]:
# Examining the shape of the sessions dataframe after applying the accuracy filter.
print(f'Shape of df_sessions {df_sessions.shape}')
profile_id_count = len(df_sessions['profile_id'].unique())
exp_id_count = len(df_sessions['exp_id'].unique())
print(f"profileIdCount:{profile_id_count}")
print(f"expIdCount:{exp_id_count}")

Shape of df_sessions (14193, 10)
profileIdCount:11759
expIdCount:14193


In [34]:
# Read the lexical-decision file using tab as a delimiter.
df_lexical = pd.read_csv('D:\\Geoff\\lexical-decision.csv',delimiter='\t')

# Applying the necessary filters.
df_lexical = df_lexical.loc[(df_lexical['lexicality'] == 'W') | ((df_lexical['lexicality'] == 'I') & (df_lexical['spelling'] == 'hotels')) ]
df_lexical['lexicality'].value_counts()

lexicality
W    1538180
I         11
Name: count, dtype: int64

In [9]:
df_lexical.head()

,trial_id,exp_id,trial_order,spelling,lexicality,rt,accuracy,response,rt_adjbox,rt_zscore
0,4724970,50572,0,gust,W,926,1,W,True,NaN
2,4724972,50572,2,stripper,W,754,1,W,True,NaN
6,4724976,50572,6,unhealthful,W,1145,1,W,True,NaN
9,4724979,50572,9,panic,W,629,1,W,True,NaN
10,4724980,50572,10,vestige,W,635,1,W,True,-0.982006


In [36]:
print(f'The shape of df_lexical is {df_lexical.shape}')
exp_id_count_lexical = len(df_lexical['exp_id'].unique())
print(f"expIdCountLexical:{exp_id_count_lexical}")

The shape of df_lexical is (1538191, 10)
expIdCountLexical:21974


In [37]:
# Select the users from df_lexical based on the users present in the final filtered df_sessions
required_exp_ids = (df_lexical['exp_id'].isin(df_sessions['exp_id']))
print((df_lexical['exp_id'].isin(df_sessions['exp_id'])).value_counts())
df_lexical = df_lexical[required_exp_ids]

exp_id
True     993516
False    544675
Name: count, dtype: int64


In [38]:
print(df_lexical.shape)
exp_id_filtered_count_lexical = len(df_lexical['exp_id'].unique())

# Print the final number of unique exp_id
print(f"expIdFilteredCountLexical:{exp_id_filtered_count_lexical}")
df_lexical.head()

(993516, 10)
expIdFilteredCountLexical:14193


,trial_id,exp_id,trial_order,spelling,lexicality,rt,accuracy,response,rt_adjbox,rt_zscore
101,4742100,50804,1,unappeased,W,1591,1,W,True,NaN
102,4742101,50804,2,rollerskate,W,998,1,W,True,NaN
103,4742102,50804,3,ultra,W,743,1,W,True,NaN
105,4742104,50804,5,spud,W,1311,1,W,True,NaN
106,4742105,50804,6,ablution,W,1505,0,N,True,NaN


In [13]:
# Process the lexical dataframe to include required columns only

required_columns = ['exp_id','spelling','accuracy']
df_lexical = df_lexical[required_columns]
print(df_lexical.head())

print(f"Number of unique words:{len(df_lexical['spelling'].unique())}")


     exp_id     spelling  accuracy
101   50804   unappeased         1
102   50804  rollerskate         1
103   50804        ultra         1
105   50804         spud         1
106   50804     ablution         0
Number of unique words:60688


In [26]:
# Check the datatypes to implement any memory improvements.
print(df_lexical.dtypes)
first_value = df_lexical['spelling'].iloc[0]
print(type(first_value))

exp_id       int64
spelling    object
accuracy     uint8
dtype: object
<class 'str'>


In [15]:
# Converting the accuracy to a lower memory type as the values are binary or empty.
df_lexical['accuracy'] = df_lexical['accuracy'].astype('uint8')
print(df_lexical.dtypes)

exp_id       int64
spelling    object
accuracy     uint8
dtype: object


In [16]:
# Check the index and reset it for memory management.
print(df_lexical.index)
df_lexical.reset_index(drop=True, inplace=True)

Index([    101,     102,     103,     105,     106,     108,     109,     110,
           111,     112,
       ...
       2197219, 2197221, 2197222, 2197223, 2197224, 2197225, 2197226, 2197230,
       2197231, 2197233],
      dtype='int64', length=993516)


In [28]:
# Check for the final index and create a new file for further processing.
print(df_lexical.index)
df_lexical.to_csv("D:\\Geoff\\lexical.csv")

RangeIndex(start=0, stop=993516, step=1)


In [19]:
# Install joblib library for parallel processing
! pip install joblib

In [20]:
from joblib import Parallel, delayed

# Function to pivot a chunk of DataFrame
def pivot_chunk(chunk):
    return chunk.pivot_table(index='exp_id', columns='spelling', values='accuracy', fill_value='')

# Read CSV file in chunks
chunk_size = 10000
chunks = pd.read_csv('D:\\Geoff\\lexical.csv', chunksize=chunk_size)

# Process chunks in parallel and concatenate results
pivoted_dfs = Parallel(n_jobs=-1)(delayed(pivot_chunk)(chunk) for chunk in chunks)
final_df = pd.concat(pivoted_dfs)

# Print the final pivoted DataFrame
print(final_df.head())


<>:9: SyntaxWarning: invalid escape sequence '\G'
<>:9: SyntaxWarning: invalid escape sequence '\G'
C:\Users\gouth\AppData\Local\Temp\ipykernel_4792\3569949980.py:9: SyntaxWarning: invalid escape sequence '\G'
  chunks = pd.read_csv('D:\Geoff\lexical.csv', chunksize=chunk_size)


spelling abandonee abductor abeam abecedary abele abet abettor abhorrent  \
exp_id                                                                     
50804                                                                      
52177                                                                      
57540                                                                      
61203                                                                      
63167          1.0                                                         

spelling abidingly abjurer  ... overcapacity articulated insolvable springe  \
exp_id                      ...                                               
50804                       ...          NaN         NaN        NaN     NaN   
52177                       ...          NaN         NaN        NaN     NaN   
57540                       ...          NaN         NaN        NaN     NaN   
61203                       ...          NaN         NaN        NaN     

In [21]:
print(f'Shape of final dataframe {final_df.shape}')
print(f'Information of the final dataframe {final_df.info()}')


(14292, 60687)
<class 'pandas.core.frame.DataFrame'>
Index: 14292 entries, 50804 to 12931042
Columns: 60687 entries, abandonee to ribald
dtypes: object(60687)
memory usage: 6.5+ GB
None


In [22]:
# Define the file path for saving the DataFrame
output_file = 'D:\\Geoff\\final.csv'  # Change the file extension as needed

# Write the DataFrame to a CSV file with efficient memory usage
final_df.to_csv(output_file, sep ='\t', index=True, index_label='exp_id', chunksize=10000,)

# For Excel file:
# final_df.to_excel(output_file, index=True)

print(f"DataFrame saved to {output_file} successfully.")


<>:2: SyntaxWarning: invalid escape sequence '\G'
<>:2: SyntaxWarning: invalid escape sequence '\G'
C:\Users\gouth\AppData\Local\Temp\ipykernel_4792\3107770976.py:2: SyntaxWarning: invalid escape sequence '\G'
  output_file = 'D:\Geoff\\final.csv'  # Change the file extension as needed


DataFrame saved to D:\Geoff\final.csv successfully.


In [23]:
# Check if the values of different words are mapped correctly by changing the word in the brackets.
final_df['ablution'].value_counts()

ablution
       1712
0.0      11
1.0       3
Name: count, dtype: int64